<a href="https://colab.research.google.com/github/FS75/ict3104-team01-2023/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/mayuelala/FollowYourPose/blob/main/quick_demo.ipynb)

# 🕺🕺🕺 Follow Your Pose 💃💃💃:
# Pose-Guided Text-to-Video Generation using Pose-Free Videos

[Yue Ma*](https://mayuelala.github.io/), [Yingqing He*](https://github.com/YingqingHe), [Xiaodong Cun](http://vinthony.github.io/), [Xintao Wang](https://xinntao.github.io/), [Ying Shan](https://scholar.google.com/citations?hl=zh-CN&user=4oXBp9UAAAAJ), [Xiu Li](https://scholar.google.com/citations?user=Xrh1OIUAAAAJ&hl=zh-CN), and [Qifeng Chen](https://cqf.io)


[![Project Website](https://img.shields.io/badge/Project-Website-orange)](https://follow-your-pose.github.io/)
[![arXiv](https://img.shields.io/badge/arXiv-2304.01186-b31b1b.svg)](https://arxiv.org/abs/2304.01186)
[![GitHub](https://img.shields.io/github/stars/mayuelala/FollowYourPose?style=social)](https://github.com/mayuelala/FollowYourPose)

In [ ]:
print("hello world from ict3101 project.")

hello world from ict3101 project.


In [ ]:
print("Ciao ")

Ciao 


In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15360 MiB, 15101 MiB


# 🕺🕺🕺 Install Environment

In [1]:
# !update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
# !update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1
# !python --version
# !apt-get update
# !apt install software-properties-common
# !sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
# !apt-get install python3-pip

print('Git clone project and install requirements...')
!rm -rf ict3104-team01-2023 && git clone https://github.com/FS75/ict3104-team01-2023.git
!rm -rf mmpose && git clone https://github.com/open-mmlab/mmpose.git

Git clone project and install requirements...
Cloning into 'ict3104-team01-2023'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 198 (delta 11), reused 6 (delta 3), pack-reused 172
Receiving objects: 100% (198/198), 93.40 MiB | 32.40 MiB/s, done.
Resolving deltas: 100% (42/42), done.
Cloning into 'mmpose'...
remote: Enumerating objects: 30057, done.
remote: Counting objects: 100% (835/835), done.
remote: Compressing objects: 100% (505/505), done.
remote: Total 30057 (delta 388), reused 645 (delta 322), pack-reused 29222
Receiving objects: 100% (30057/30057), 51.73 MiB | 31.09 MiB/s, done.
Resolving deltas: 100% (21184/21184), done.


In [ ]:
!cd /content/ict3104-team01-2023/FollowYourPoseTeam1
!export PYTHONPATH=/content/ict3104-team01-2023/FollowYourPoseTeam1:$PYTHONPATH
# %pip install -r requirements.txt
!python3.10 -m pip install -q -U --pre triton
!apt update
!python3.10 -m pip install -q diffusers==0.11.1 torch==1.13.1 transformers==4.26.0 bitsandbytes==0.35.4 \
imageio-ffmpeg xformers==0.0.16  --extra-index-url https://download.pytorch.org/whl/cu113


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.0.1+cu118 requires triton==2.0.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 2.1.0 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [517 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http:/

In [ ]:
%cd /content/ict3104-team01-2023/FollowYourPoseTeam1
!python3.10 -m pip install -r requirements.txt

/content/ict3104-team01-2023/FollowYourPoseTeam1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

# 💃💃💃 Weight

In [ ]:
%mkdir checkpoints
!git lfs install
!git clone https://huggingface.co/YueMafighting/FollowYourPose_v1
%mv /content/ict3104-team01-2023/FollowYourPoseTeam1/FollowYourPose_v1/* /content/ict3104-team01-2023/FollowYourPoseTeam1/checkpoints/
%rm -rf FollowYourPose_v1

Updated git hooks.
Git LFS initialized.
Cloning into 'FollowYourPose_v1'...
remote: Enumerating objects: 42, done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 42
Unpacking objects: 100% (42/42), 584.62 KiB | 2.02 MiB/s, done.
Filtering content: 100% (9/9), 9.75 GiB | 46.88 MiB/s, done.


# 🕺🕺🕺 Sample

Due to memory of GPU, we recommend set video_length=8 in ./config/pose_sample.yaml for running successfully.

Meanwhile, we should keep the skeleton frame length(./followyourpose/pipelines/pipeline_followyourpose.py:422 ) equal with video_length

In [ ]:
%cd /content/ict3104-team01-2023/FollowYourPoseTeam1
!pwd
!TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py --config="configs/pose_sample.yaml"  --skeleton_path="./pose_example/vis_ikun_pose2.mov"

100% 50/50 [03:43<00:00,  4.47s/it]
Saving videos now
Video loaded successfully: /content/ict3104-team01-2023/FollowYourPoseTeam1/checkpoints/inference/sample-1000-33-2023-10-01 13:14:40.759687/Stormtrooper on the sea.mp4
Video with caption saved as "/content/ict3104-team01-2023/FollowYourPoseTeam1/checkpoints/inference/sample-1000-33-2023-10-01 13:14:40.759687/Stormtrooper on the sea_with_captions.mp4"
100% 50/50 [03:49<00:00,  4.59s/it]
Saving videos now
Video loaded successfully: /content/ict3104-team01-2023/FollowYourPoseTeam1/checkpoints/inference/sample-1000-33-2023-10-01 13:14:40.759687/Astronaut on the beach.mp4
Video with caption saved as "/content/ict3104-team01-2023/FollowYourPoseTeam1/checkpoints/inference/sample-1000-33-2023-10-01 13:14:40.759687/Astronaut on the beach_with_captions.mp4"


# 💃💃💃 The videos are output into ./checkpoints/inference/sample-xxx/<prompt>.gif

##Data Exploration

In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
from moviepy.editor import *
from IPython.display import clear_output

def play_video(change):
    clear_output(wait=True)  # clear the current cell output before displaying new video
    if video_dropdown.value:
        video_path = os.path.join('/content/ict3104-team01-2023/data/videos', video_dropdown.value)
        video_clip = VideoFileClip(video_path)
        # Display the video
        display(video_dropdown)  # re-display the dropdown
        display(play_button)  # re-display the play button
        display(video_clip.ipython_display(width=480))
    else:
        print("Please select a video")


# List the video files
video_files = os.listdir('/content/ict3104-team01-2023/data/videos')

# Create a dropdown menu
video_dropdown = widgets.Dropdown(
    options=video_files,
    description='Select Video:',
)
# Create a button to play the video
play_button = widgets.Button(description="Play Video")

# Attach the play_video function to the button's on_click event
play_button.on_click(play_video)

# Display the dropdown menu and play button
display(video_dropdown)
display(play_button)

Dropdown(description='Select Video:', index=5, options=('0DDYA.mp4', '0CVZN.mp4', '0CG15.mp4', '0A8ZT.mp4', '0…

Button(description='Play Video', style=ButtonStyle())

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



t:  98%|█████████▊| 974/996 [00:02<00:00, 328.35it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/ict3104-team01-2023/data/videos/0CX32.mp4, 388800 bytes wanted but 0 bytes read,at frame 995/996, at time 33.20/33.23 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready __temp__.mp4


# Inference

## Selecting a pre-trained model

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import os

# Get the current working directory
# current_working_directory = os.getcwd()

# Print the current working directory
# print(current_working_directory)

# Assuming your folder path is '/
folder_path = 'ict3104-team01-2023/data/pre_trained_models'

# List all files in the folder
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Create a dropdown menu with the file names
file_selector = widgets.Dropdown(
    options=files,
    description='Select Model:',
    disabled=False,
)

# Display the dropdown menu
display(file_selector)

# Now, the selected file can be accessed using file_selector.value

Dropdown(description='Select Model:', options=('abc.txt',), value='abc.txt')

# For Input

In [ ]:
#@markdown Select video After running the code
import os
import ipywidgets as widgets
from IPython.display import display, HTML
directory_path = '/content/ict3104-team01-2023/data/videos'
mp4_files = [f for f in os.listdir(directory_path) if f.endswith('.mp4')]

dropdown = widgets.Dropdown(
    options={f: os.path.join(directory_path, f) for f in mp4_files},
    description='Select video:',
    disabled=False,
)
def update_link(change):
    selected_mp4_file  = dropdown.value
    link = f'<a href="{selected_mp4_file}" target="_blank">{selected_mp4_file}</a>'
    link_output.value = link

dropdown.observe(update_link, 'value')

# Display the dropdown widget
display(dropdown)

# Create an output widget to display the selected directory as a link
link_output = widgets.Output()
display(link_output)

#@markdown Enter the things to show in the video
TEXT_Prompts = "" #@param {type:"string"}


Dropdown(description='Select video:', options={'0BX9N.mp4': '/content/ict3104-team01-2023/data/videos/0BX9N.mp…

Dropdown(description='Select File:', options=('project.ipynb', '.gitignore', 'README.md'), value='project.ipyn…